In [78]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..")
import d2lzh_pytorch.torch as d2l
from d2lzh_pytorch.utils import FlattenLayer
from d2lzh_pytorch.utils import train_ch5
from d2lzh_pytorch.CNN.VGG_model import get_VGG_model


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.__version__)

1.13.1+cu117


In [52]:
# 数据加载

batch_size = 256
#rootPath = r"/mnt/g1t/ai_data/Datasets_on_HHD/FashionMNIST"
#train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, root=rootPath)

rootPath = r"/mnt/g1t/ai_data/Datasets_on_HHD/FashionMNIST"
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, 
                                                    resize=224,
                                                    root =rootPath 
                                                   )

In [80]:
# 加载已训练好的模型
m_name = "VGG_net_2023_06_27_00-27-09.pt"  # 7.8M

model = torch.load(m_name)

VGG = get_VGG_model()
VGG.load_state_dict(model.state_dict())

<All keys matched successfully>

In [93]:
VGG

Sequential(
  (vgg_block_1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_4): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [91]:
def predict_ch3(net, test_iter, n=6):  #@save
    """预测标签（定义见第3章）"""
    
    err = 0
    succ = 0
    
    for X, y in test_iter:
    
        trues = d2l.get_fashion_mnist_labels(y)
        preds = d2l.get_fashion_mnist_labels(net(X).argmax(axis=1))  # 计算结果, 预测结果
        titles = [true +'\n' + pred for true, pred in zip(trues, preds)]

        X_resized = F.interpolate(X[0: n], size=(28, 28), mode='nearest')
        # print(X_resized.shape)   # 输出：torch.Size([6, 1, 28, 28])
        
        for true, pred in zip(trues, preds):
            
            if true != pred:
                # print("判断错误: 标签是:%s \t 计算为:%s" % (true, pred))
                err += 1
            else:
                succ += 1
                pass
        # d2l.show_images(X_resized.reshape((n, 28, 28)), 1, n, titles=titles[0:n])
        # break
        pass
    
    succ_per = succ / (succ + err)
    print("正确 %s, 错误: %s, 正确率: %s" % (succ, err, succ_per))

In [92]:
test_n = 6


predict_ch3(VGG,
            test_iter,
            n=test_n)

正确 9104, 错误: 896, 正确率: 0.9104
